In [51]:
#Import packages
from polygon.rest import RESTClient
import requests
import pandas as pd
import os
import numpy as np
import datetime as dt

#Import polygon API key from file
polygon_api_key = open("polygon_io_api_key.txt").read()
alphavantage_api_key = open("alphavantage_api_key.txt").read()

client = RESTClient("polygon_api_key")

ticker = "intu".upper()

url = f"https://api.polygon.io/benzinga/v1/earnings?ticker={ticker}&limit=100&sort=date.desc&apiKey={polygon_api_key}&date_status=confirmed"
r = requests.get(url)
data = r.json()

df_len = []
for i in range(len(data["results"])):
    df_len.append(i)


df = pd.DataFrame(data["results"], index=df_len)
df_column_drop = ['currency','date_status','benzinga_id','importance','company_name','estimated_revenue','previous_revenue','last_updated','eps_surprise','eps_method','actual_revenue','revenue_surprise','revenue_method']
df.drop(df_column_drop, axis='columns', inplace=True)

# Calls the API to get earnings data
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={alphavantage_api_key}'
r = requests.get(url)
earnings = r.json()

# Extract quarterly earnings data
quarterly_earnings = earnings['quarterlyEarnings']

# Convert quarterly earnings data into a DataFrame
quarterly_earnings_df = pd.DataFrame(quarterly_earnings)

last_earnings = quarterly_earnings_df[["reportedDate","reportTime"]]

alpha_earnings = last_earnings

benzinga_earnings = df[["date","time"]]

benzinga_earnings.loc[:, "time"] = pd.to_datetime(benzinga_earnings["time"], format='%H:%M:%S').dt.time

#print(benzina_earnings["time"])
noon = dt.time(12,0,0)
market_periods = []

for time in benzinga_earnings["time"]:
    if time >= noon:
        market_periods.append('post-market')
    else:
        market_periods.append('pre-market')

benzinga_earnings.loc[:,'market_period'] = market_periods
benzinga_earnings = benzinga_earnings.drop('time',axis=1)


merged_df = pd.merge(benzinga_earnings, alpha_earnings, left_index=True, right_index=True, how='inner')
merged_df.loc[:,'date_equal'] = merged_df['date'] == merged_df['reportedDate'] 
merged_df.loc[:,'time_equal'] = merged_df['market_period'] == merged_df['reportTime']
merged_df.loc[:,'total_equal'] = merged_df['date_equal'] & merged_df['time_equal']
merged_df_true = merged_df[merged_df['total_equal'] == False]
print(merged_df_true)


Empty DataFrame
Columns: [date, market_period, reportedDate, reportTime, date_equal, time_equal, total_equal]
Index: []


C:\Users\Brandon\AppData\Local\Temp\ipykernel_24580\1946213777.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benzinga_earnings.loc[:,'market_period'] = market_periods
